# Spectrogram-based resnet digit classifier using fastai

In [1]:
%matplotlib inline
from pathlib import Path
import librosa
from librosa import display
import IPython.display
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import multiprocessing as mp
import matplotlib.pyplot as plt

In [2]:
# Path to data directory and recordings directory
DATA = Path("../../data/words/")
RECORDINGS = DATA/"audio-recordings"

In [3]:
def load_wav(filename):
    return librosa.core.load(filename, sr=None, mono=True)

In [4]:
labels_df = pd.read_csv(DATA/"labels.csv")

In [5]:
def save_spectrogram(wav_path, save_path):
    plt.clf()
    data, sr = load_wav(wav_path)
    trimmed, idx = librosa.effects.trim(data, top_db=30)
    spec = librosa.feature.melspectrogram(trimmed, sr, n_fft=2048, hop_length=256)
    librosa.display.specshow(librosa.core.power_to_db(spec))
    cur_axes = plt.gca().set_axis_off()
    plt.savefig(save_path, bbox_inches="tight", pad_inches=0)

In [6]:
def process_file(fname):
    save_spectrogram(DATA/"processed"/fname, DATA/"spectrograms"/fname.replace(".wav", ".png"))

In [7]:
!rm ../../data/words/spectrograms/*.png

if not (DATA/"spectrograms").exists():
    (DATA/"spectrograms").mkdir()

In [8]:
pool = mp.Pool(4)  # Use 4 processes

for fname in tqdm(labels_df["filename"]):
    pool.apply(process_file, args=(fname,))
    
pool.close()

---

# Train models

In [7]:
from fastai.vision import *
from fastai import metrics
import torchvision

In [8]:
# Create new dataframe which has filenames ending in .png instead of .wav
spec_labels_df = labels_df.copy()
spec_labels_df["filename"] = spec_labels_df["filename"].apply(lambda x: x.replace(".wav", ".png"))

In [9]:
def get_data(valid_split_no, bs=32):
    df = spec_labels_df[spec_labels_df[f"valid{valid_split_no}"] != -1]
    return (ImageList.from_df(df, DATA/"spectrograms")
            .split_from_df(col=f"valid{valid_split_no}")
            .label_from_df(cols="label")
            .databunch(bs=bs)
           )

In [10]:
VALID_SPLITS = [1, 2, 3]
MODELS = [
    #models.resnet18,
    #models.resnet34,
    models.resnet50,
    #models.resnet101,
    models.densenet121,
    models.densenet161
]
PRETRAINED = [True, False]
USE_MIXUP = [True, False]

In [ ]:
for valid_split in VALID_SPLITS:
    print(f"************************************\nSPLIT {valid_split}\n************************************")
    for model in MODELS:
        for pretrained in PRETRAINED:
            for use_mixup in USE_MIXUP:
                # Free up GPU memory
                print(f"model={model.__name__}, pretrained={pretrained}, mixup={use_mixup}")
                gc.collect()
                torch.cuda.empty_cache()

                learn = cnn_learner(get_data(valid_split, bs=32),
                                    model,
                                    pretrained=pretrained,
                                    metrics=[accuracy, metrics.Precision(average="macro"), metrics.Recall(average="macro")],
                                    # Using macro average precision and recall
                                    callback_fns=[ShowGraph,
                                                  partial(callbacks.CSVLogger, filename=f"Split{valid_split}_{model.__name__},pretrained={int(pretrained)},mixup={int(use_mixup)}")
                                                 ]).to_fp16()
                if use_mixup:
                    learn = learn.mixup()

                learn.fit_one_cycle(100, max_lr=1e-2)
                learn.export(f"Split{valid_split}_{model.__name__},pretrained={int(pretrained)},mixup={int(use_mixup)}")
                del learn

************************************
SPLIT 1
************************************
model=resnet50, pretrained=True, mixup=True
epoch     train_loss  valid_loss  accuracy  precision  recall    time    
0         5.578439    4.533839    0.041667  nan        0.041667  00:21     
1         4.934674    4.355747    0.063889  nan        0.063889  00:20     
2         4.508043    4.250220    0.056944  nan        0.056944  00:20     
3         4.159351    4.277453    0.065278  nan        0.065278  00:20     
4         3.976144    4.297225    0.065278  nan        0.065278  00:20     
5         3.803868    4.341218    0.068056  nan        0.068056  00:20     
6         3.760157    4.212197    0.066667  nan        0.066667  00:20     
7         3.691179    4.235294    0.073611  nan        0.073611  00:20     
8         3.588504    4.182394    0.088889  nan        0.088889  00:20     
9         3.565894    4.393504    0.102778  nan        0.102778  00:20     
10        3.579708    4.246653    0.1041